In [1]:
import pandas as pd
from wordcloud import STOPWORDS


In [2]:
#add some unnecessary words to STOPWORDS list
STOPWORDS.add("rt")
STOPWORDS.add("s")
STOPWORDS.add("u")
STOPWORDS.add("amp")
STOPWORDS.add("th")
STOPWORDS.add("will")
STOPWORDS.add("t")
STOPWORDS.add("m")
STOPWORDS.add("ha")
STOPWORDS.add("wa")

In [3]:
df=pd.read_pickle("corpus.pkl")

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train, test = train_test_split(df, test_size=0.3, train_size=0.7, random_state=14)
train.shape, test.shape

((60521, 5), (25938, 5))

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.lancaster import LancasterStemmer
import nltk

In [7]:
st = LancasterStemmer()
def token(text):
    txt = nltk.word_tokenize(text.lower())
    return [st.stem(word) for word in txt]

In [8]:
cv = CountVectorizer(tokenizer=token,stop_words=None,
                     analyzer=u'word', min_df=4)
X_train_cv = cv.fit_transform(train['clean_headline']) # fit_transform learns the vocab and one-hot encodes
X_test_cv = cv.transform(test['clean_headline']) # transform uses the same vocab and one-hot encodes
# print the dimensions of the training set (text messages, terms)
print(X_train_cv.shape)

(60521, 8876)


In [17]:
cv = CountVectorizer(stop_words=None,
                     analyzer=u'word', min_df=4)
X_train_cv = cv.fit_transform(train['clean_headline']) # fit_transform learns the vocab and one-hot encodes
X_test_cv = cv.transform(test['clean_headline']) # transform uses the same vocab and one-hot encodes
# print the dimensions of the training set (text messages, terms)
print(X_train_cv.shape)

(60521, 12094)


In [18]:
count = open("cv.pkl","wb")
joblib.dump(cv,count)
count.close

<function BufferedWriter.close>

In [9]:
X_train_cv = pd.DataFrame(X_train_cv.toarray(), columns=cv.get_feature_names())
X_test_cv = pd.DataFrame(X_test_cv.toarray(), columns=cv.get_feature_names())

In [10]:
X_train_cv

,aaas,aaf,aajc,aan,aap,aapiequalpay,aapiheritagemon,aaron,aarp,aarpadvoc,...,zink,zion,zip,zoe,zon,zte,zuckerb,zuckerberg,zuckerberghear,zulu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

### Naive Bayes

In [12]:
nb = MultinomialNB()
# Train the model
nb.fit(X_train_cv, train['Party_log'])
# Take the model that was trained on the X_train_cv data and apply it to the X_test_cv data
y_pred_cv_nb = nb.predict(X_test_cv)
y_pred_cv_nb

array([0, 1, 1, ..., 0, 0, 1], dtype=int64)

In [ ]:
accuracy_score(y_pred_cv_nb, test['Party_log'])

### Logistic Regression

In [67]:
 # Use a logistic regression model
lr = LogisticRegression()
# Train the model
lr.fit(X_train_cv,train['Party_log'])
# Take the model that was trained on the X_train_cv data and apply it to the X_test_cv data
y_pred_cv = lr.predict(X_test_cv)
y_pred_cv

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0, 1, 1, ..., 0, 0, 1], dtype=int64)

In [68]:
accuracy_score(y_pred_cv, test['Party_log'])

0.787724573984116

### RandomForest

In [74]:
rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rfc.fit(X_train_cv,train['Party_log'])

y_pred_rfc=rfc.predict(X_test_cv)
accuracy_score(y_pred_rfc, test['Party_log'])

0.7545685866296553

### AdaBoost

In [ ]:
ac = AdaBoostClassifier(n_estimators = 100)
ac.fit(X_train_cv, train['Party_log'])

y_pred_ac = ac.predict(X_test_cv)
accuracy_score(y_pred_ac, test['Party_log'])

### DecisionTrees


In [21]:
dtc = DecisionTreeClassifier(random_state = 42)
dtc.fit(X_train_cv, train['Party_log'])

y_pred_dtc = dtc.predict(X_test_cv)
accuracy_score(y_pred_dtc, test['Party_log'])


0.7014033464415144

### Comparing the accuracy for individual labels

In [ ]:
import scipy 
import spicy.sparse as sp
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_predict

In [71]:
print(classification_report(y_pred_cv_nb,test['Party_log']))

              precision    recall  f1-score   support

           0       0.81      0.78      0.79     13910
           1       0.75      0.79      0.77     12028

    accuracy                           0.78     25938
   macro avg       0.78      0.78      0.78     25938
weighted avg       0.78      0.78      0.78     25938



In [ ]:
print(classification_report(y_pred_rfc,test['Party_log']))

In [ ]:
print(classification_report(y_pred_ac,test['Party_log']))

In [ ]:
print(classification_report(y_pred_dtc,test['Party_log']))

In [14]:
 import joblib

In [23]:
decisiontre = open("decision.pkl","wb")
joblib.dump(dtc,decisiontre)
decisiontre.close

<function BufferedWriter.close>